In [ ]:
import numpy as np 
import os
import cv2

In [ ]:
from model import DepthEstimate
model = DepthEstimate()
batch_size     = 16
learning_rate  = 0.0001
epochs         = 6

In [ ]:
import tensorflow as tf

from io import BytesIO
from zipfile import ZipFile
from sklearn.utils import shuffle

class DataLoader():
    def __init__(self, csv_file='data/nyu2_train.csv', DEBUG=False):
        self.shape_rgb = (480,640, 3)
        self.shape_depth = (240,320, 1)
        self.read_nyu_data(csv_file, DEBUG=DEBUG)
    def read_nyu_data(self, csv_file, DEBUG=False):
        csv = open(csv_file, 'r').read()
        nyu2_train = list((row.split(',') for row in (csv).split('\n') if len(row) > 0))

        # Dataset shuffling happens here
        nyu2_train = shuffle(nyu2_train, random_state=0)

        # Test on a smaller dataset
        if DEBUG: nyu2_train = nyu2_train[:10]
        
        # A vector of RGB filenames.
        self.filenames = [i[0] for i in nyu2_train]

        # A vector of depth filenames.
        self.labels = [i[1] for i in nyu2_train]

        # Length of dataset
        self.length = len(self.filenames)

    def _parse_function(self, filename, label): 
        # Read images from disk
        #image_decoded = tf.image.decode_jpeg(tf.io.read_file(filename))
        depth_resized = tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(label)), [self.shape_depth[0], self.shape_depth[1]])
        image_decoded = tf.image.resize(tf.image.decode_jpeg(tf.io.read_file(filename)), [self.shape_rgb[0], self.shape_rgb[1]])
        # Format
        rgb = tf.image.convert_image_dtype(image_decoded, dtype=tf.float32)
        depth = tf.image.convert_image_dtype(depth_resized / 255.0, dtype=tf.float32)
        
        # Normalize the depth values (in cm)
        depth = 1000 / tf.clip_by_value(depth * 1000, 10, 1000)

        return rgb, depth

    def get_batched_dataset(self, batch_size):
        self.dataset = tf.data.Dataset.from_tensor_slices((self.filenames, self.labels))
        self.dataset = self.dataset.shuffle(buffer_size=len(self.filenames), reshuffle_each_iteration=True)
        self.dataset = self.dataset.repeat()
        self.dataset = self.dataset.map(map_func=self._parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        #self.dataset = self.dataset.map(map_func=self.nyu_resize)
        self.dataset = self.dataset.batch(batch_size=batch_size)

        return self.dataset

In [ ]:
dl = DataLoader()
train_generator = dl.get_batched_dataset(batch_size)

print('Data loader ready.')

In [ ]:
import tensorflow.keras.backend as K

def depth_loss_function(y_true, y_pred, theta=0.3, maxDepthVal=1000.0/10.0):
    
    # Point-wise depth
    l_depth = K.mean(K.abs(y_pred - y_true), axis=-1)

    # Structural similarity (SSIM) index
    l_ssim = K.clip((1 - tf.image.ssim(y_true, y_pred, maxDepthVal)) * 0.5, 0, 1)

    # Weights
    w1 = 1.0
    w3 = theta

    return (w1 * l_ssim)  + (w3 * K.mean(l_depth))

In [ ]:
import tensorflow
#from loss import depth_loss_function

optimizer = tensorflow.keras.optimizers.Adam(lr=learning_rate, amsgrad=True)

model.compile(loss=depth_loss_function, optimizer=optimizer)

In [ ]:
import numpy as np
from io import BytesIO
rgb = np.load('eigen_test_rgb.npy')
depth = np.load('eigen_test_depth.npy')
crop = np.load('eigen_test_crop.npy')
print('Test data loaded.\n')

In [ ]:
model.load_weights("~/cp.ckpt")

In [ ]:
from utils import predict, load_images, display_images, evaluate

In [ ]:
import time

b=time.time()
e = evaluate(model, rgb, depth, crop, batch_size=1)
print("Pretrained Weights - eval results ")
print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rms', 'log_10'))
print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(e[0],e[1],e[2],e[3],e[4],e[5]))
print("eval time...")
print(time.time()-b)


In [ ]:
class ev_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    b=time.time()
    e = evaluate(model, rgb1, depth1, crop, batch_size=1)
    print("eval results ")
    print("{:>10}, {:>10}, {:>10}, {:>10}, {:>10}, {:>10}".format('a1', 'a2', 'a3', 'rel', 'rms', 'log_10'))
    print("{:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}, {:10.4f}".format(e[0],e[1],e[2],e[3],e[4],e[5]))
    print("eval time...")
    print(time.time()-b)

In [ ]:

import os
import tensorflow

checkpoint_path = "~pathto store checkpoints/cp.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)

history=model.fit(train_generator, epochs=epochs, steps_per_epoch=dl.length//batch_size, callbacks=[cp_callback,ev_callback()],initial_epoch=4)
